In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# Setup WebDriver
options = Options()
options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Open Trustpilot page
url = "https://www.trustpilot.com/review/www.zendesk.com"
driver.get(url)

# Initialize reviews list
all_reviews = []

while True:
    try:
        # Wait for reviews to load
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.XPATH, "//p[contains(@class, 'typography_body')]"))
        )

        # Extract reviews on this page
        review_elements = driver.find_elements(By.XPATH, "//p[contains(@class, 'typography_body')]")
        page_reviews = [elem.text for elem in review_elements]
        all_reviews.extend(page_reviews)

        # Try clicking the next page button
        next_button = driver.find_element(By.CSS_SELECTOR, 'a[data-pagination-button-next-link]')
        if 'disabled' in next_button.get_attribute('class'):
            break  # No more pages
        next_button.click()
        time.sleep(2)  # Give time for page transition

    except Exception as e:
        print("No more pages or error:", e)
        break

# Convert to DataFrame
df_reviews = pd.DataFrame(all_reviews, columns=["review"])
print(df_reviews.head())
print(f"✅ Total reviews scraped: {len(df_reviews)}")

driver.quit()

No more pages or error: Message: stale element reference: stale element not found
  (Session info: chrome-headless-shell=120.0.6099.129); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
0   chromedriver                        0x000000010321c4dc chromedriver + 4162780
1   chromedriver                        0x0000000103214664 chromedriver + 4130404
2   chromedriver                        0x0000000102e6bbc0 chromedriver + 293824
3   chromedriver                        0x0000000102e71234 chromedriver + 315956
4   chromedriver                        0x0000000102e7313c chromedriver + 323900
5   chromedriver                        0x0000000102e73278 chromedriver + 324216
6   chromedriver                        0x0000000102eb8fec chromedriver + 610284
7   chromedriver                        0x0000000102eb7090 chromedriver + 602256
8   chromedriver                        0x00000

In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# Setup WebDriver
options = Options()
options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Open Trustpilot page
url = "https://www.trustpilot.com/review/www.zendesk.com"
driver.get(url)

# Initialize reviews list
all_reviews = []

while True:
    try:
        # Wait for reviews to load
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.XPATH, "//p[contains(@class, 'typography_body')]"))
        )

        # Extract reviews on this page
        review_elements = driver.find_elements(By.XPATH, "//p[contains(@class, 'typography_body')]")
        page_reviews = [elem.text for elem in review_elements]
        all_reviews.extend(page_reviews)

        # Try clicking the next page button
        next_button = driver.find_element(By.CSS_SELECTOR, 'a[data-pagination-button-next-link]')
        if 'disabled' in next_button.get_attribute('class'):
            break  # No more pages
        next_button.click()
        time.sleep(2)  # Give time for page transition

    except Exception as e:
        print("No more pages or error:", e)
        break

# Convert to DataFrame
df_reviews = pd.DataFrame(all_reviews, columns=["review"])
print(df_reviews.head())
print(f"✅ Total reviews scraped: {len(df_reviews)}")

driver.quit()

No more pages or error: Message: stale element reference: stale element not found
  (Session info: chrome-headless-shell=120.0.6099.129); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
0   chromedriver                        0x00000001052fc4dc chromedriver + 4162780
1   chromedriver                        0x00000001052f4664 chromedriver + 4130404
2   chromedriver                        0x0000000104f4bbc0 chromedriver + 293824
3   chromedriver                        0x0000000104f51234 chromedriver + 315956
4   chromedriver                        0x0000000104f5313c chromedriver + 323900
5   chromedriver                        0x0000000104f53278 chromedriver + 324216
6   chromedriver                        0x0000000104f98fec chromedriver + 610284
7   chromedriver                        0x0000000104f97090 chromedriver + 602256
8   chromedriver                        0x00000

In [3]:
pip install psycopg2-binary

Note: you may need to restart the kernel to use updated packages.


In [9]:
from sqlalchemy import create_engine

# PostgreSQL Connection Details
user = 'postgres'
password = 'isba_4715'
host = 'isba-dev-02.cxue0o466fzu.us-east-1.rds.amazonaws.com'
port = 5432
database = 'postgres'  # Change this if using a different DB name

# Create engine string
engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}')

# Upload DataFrame
df_reviews.to_sql(name='trustpilot_reviews', con=engine, if_exists='replace', index=False)

print("✅ Data uploaded to PostgreSQL.")

✅ Data uploaded to PostgreSQL.


In [11]:
df_reviews.to_sql(
    name='trustpilot_reviews',
    con=engine,
    schema='sql_project',  # 👈 this ensures it lands in the correct schema
    if_exists='replace',
    index=False
)

98

In [2]:
from dotenv import load_dotenv
import os
import pandas as pd
from sqlalchemy import create_engine

load_dotenv()

pg_conn_str = f"postgresql+psycopg2://{os.getenv('PG_USER')}:{os.getenv('PG_PASSWORD')}@{os.getenv('PG_HOST')}/{os.getenv('PG_DB')}"
pg_engine = create_engine(pg_conn_str)

# Extract
query = "SELECT * FROM sql_project.trustpilot_reviews"
df = pd.read_sql(query, pg_engine)

# Load
df.to_sql(
    name='trustpilot_reviews_raw',
    con=pg_engine,
    schema='raw',
    if_exists='replace',
    index=False
)

print("✅ Successfully extracted and loaded Trustpilot Reviews!")


✅ Successfully extracted and loaded Trustpilot Reviews!


In [4]:
# Quick checks
print(df.info())
print(df.isnull().sum())
print(df.describe())
print(df.head())

# Light cleaning examples
df.columns = [col.strip().lower().replace(' ', '_') for col in df.columns]  # Clean column names
df = df.drop_duplicates()  # Remove duplicate rows
df = df.dropna(subset=['review'])  # Remove rows missing important fields



<class 'pandas.core.frame.DataFrame'>
Index: 88 entries, 0 to 91
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   review  88 non-null     object
dtypes: object(1)
memory usage: 1.4+ KB
None
review    0
dtype: int64
       review
count      88
unique     88
top       1.8
freq        1
                                              review
0                                                1.8
1  Companies on Trustpilot can’t offer incentives...
2  the tool is easy to use, which allows to make ...
3  Zendesk, the AI only support company charging ...
4  Warning .This company accessed my credit card ...
